In [1]:
import sys, os
import pickle

import pandas as pd
from itertools import combinations

from github import Github

sys.path.append(os.path.abspath('package'))

from package.graph import KnowledgeGraphBuilder

C:\Users\szmid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Build knowledge graph

In [2]:
with open ('./_/git_token.txt', 'r') as f:
    git_token = f.read().strip()

kgb = KnowledgeGraphBuilder(git_token)

In [ ]:
# Only create the graph to the feature extraction repository for faster workflow and visualization
repograph = kgb.build_knowledge_graph(
    repo_name='scikit-learn/scikit-learn',
    graph_type='CFG',
    num_of_PRs=3,
    repo_path_modifier='sklearn/manifold'
)

In [4]:
repograph.keys()

dict_keys(['function_nodes', 'function_edges', 'subgraph_nodes', 'subgraph_edges', 'subgraph_function_edges', 'function_subgraph_edges', 'import_nodes', 'import_function_edges', 'pr_nodes', 'pr_function_edges', 'issue_nodes', 'issue_pr_edges', 'artifacts', 'actions'])

In [13]:
repograph['actions']

,name,path,triggers,platforms,actions_used
0,Unassign,.github/workflows/unassign.yml,None,[ubuntu-latest],[]
1,Wheel builder,.github/workflows/wheels.yml,None,"[ubuntu-latest, ${{ matrix.os }}]","[conda-incubator/setup-miniconda@v3, actions/c..."
2,Pull Request Labeler,.github/workflows/labeler-module.yml,None,[ubuntu-latest],[thomasjpfan/labeler@v2.5.1]
3,Pull Request Regex Title Labeler,.github/workflows/labeler-title-regex.yml,None,[ubuntu-24.04],"[actions/checkout@v4, actions/setup-python@v5]"
4,Publish to Pypi,.github/workflows/publish_pypi.yml,None,[ubuntu-latest],[pypa/gh-action-pypi-publish@76f52bc884231f62b...
5,Check Changelog,.github/workflows/check-changelog.yml,None,[ubuntu-latest],"[actions/checkout@v4, scientific-python/action..."
6,Codespell,.github/workflows/codespell.yml,None,[ubuntu-latest],"[codespell-project/actions-codespell@v2, actio..."
7,Labels Blank issues,.github/workflows/label-blank-issue.yml,None,[ubuntu-latest],[andymckay/labeler@1.0.4]
8,Update tracking issue,.github/workflows/update_tracking_issue.yml,None,[ubuntu-latest],"[actions/checkout@v4, actions/setup-python@v5]"
9,CircleCI artifacts redirector,.github/workflows/artifact-redirector.yml,None,[ubuntu-latest],[scientific-python/circleci-artifacts-redirect...


## Save and load file

In [ ]:
with open('./_/sklearn_new.pkl', 'wb') as f:
    pickle.dump(repograph, f)

In [14]:
with open('_/sklearn_new.pkl', 'rb') as f:
    repograph = pickle.load(f)

## Visualize graph

In [12]:
kgb.visualize_graph(repograph, show_subgraph_nodes=False)

Filtered sklearn graph. visualization saved to filtered_graph.html


## Save to Neo4j

In [14]:
kgb.store_knowledge_graph_in_neo4j(
    "neo4j://127.0.0.1:7687",
    "neo4j",
    "password",
    repograph
)